In [1]:
import networkx as nx
import random
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from typing import List
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from karateclub.utils.treefeatures import WeisfeilerLehmanHashing
from sklearn.cluster import KMeans
from kneed import KneeLocator
from scipy.spatial.distance import cdist
import pymysql
import base64
from itertools import combinations
import umap
from mpl_toolkits.mplot3d import Axes3D
from scipy.cluster.hierarchy import linkage, fcluster, cophenet, dendrogram
from scipy.spatial.distance import pdist
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.cluster import KMeans, AgglomerativeClustering

**Graph2vec and UMAP**

In [2]:
class Graph2Vec:
    def __init__(self, wl_iterations: int = 2, attributed: bool = False, dimensions: int = 128,
                 workers: int = 4, down_sampling: float = 0.0001, epochs: int = 10, learning_rate: float = 0.025,
                 min_count: int = 5, seed: int = 42, erase_base_features: bool = False):
        self.wl_iterations = wl_iterations
        self.attributed = attributed
        self.dimensions = dimensions
        self.workers = workers
        self.down_sampling = down_sampling
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.min_count = min_count
        self.seed = seed
        self.erase_base_features = erase_base_features

    def _set_seed(self):
        np.random.seed(self.seed)

    def fit(self, graphs: List[nx.classes.graph.Graph]):
        self._set_seed()
        documents = [
            WeisfeilerLehmanHashing(
                graph, self.wl_iterations, self.attributed, self.erase_base_features
            )
            for graph in graphs
        ]
        documents = [
            TaggedDocument(words=doc.get_graph_features(), tags=[str(i)])
            for i, doc in enumerate(documents)
        ]

        self.model = Doc2Vec(
            documents,
            vector_size=self.dimensions,
            window=0,
            min_count=self.min_count,
            dm=0,
            sample=self.down_sampling,
            workers=self.workers,
            epochs=self.epochs,
            alpha=self.learning_rate,
            seed=self.seed,
        )

        self._embedding = [self.model.docvecs[str(i)] for i, _ in enumerate(documents)]

    def get_embedding(self) -> np.array:
        return np.array(self._embedding)

    def infer(self, graphs) -> np.array:
        self._set_seed()
        documents = [
            WeisfeilerLehmanHashing(
                graph, self.wl_iterations, self.attributed, self.erase_base_features
            )
            for graph in graphs
        ]
        documents = [TaggedDocument(words=doc.get_graph_features(), tags=[str(i)]) for i, doc in enumerate(documents)]
        return np.array([self.model.infer_vector(doc.words) for doc in documents])


In [3]:
class Graph2VecUMAP:
    def __init__(self, graph2vec: Graph2Vec, n_neighbors: int = 5, min_dist: float = 0.1,
                 n_components: int = 4, metric: str = 'euclidean', random_state: int = 42):
        self.graph2vec = graph2vec
        self.n_neighbors = n_neighbors
        self.min_dist = min_dist
        self.n_components = n_components
        self.metric = metric
        self.random_state = random_state

    def fit_transform(self, graphs: List[nx.classes.graph.Graph]):
        graph2vec_embeddings = self.graph2vec.infer(graphs)
        self.umap_model = umap.UMAP(
            n_neighbors=self.n_neighbors,
            min_dist=self.min_dist,
            n_components=self.n_components,
            metric=self.metric,
            random_state=self.random_state
        )
        self.embedding = self.umap_model.fit_transform(graph2vec_embeddings)
        return self.embedding

    def transform(self, graphs: List[nx.classes.graph.Graph]):
        graph2vec_embeddings = self.graph2vec.infer(graphs)
        self.embedding = self.umap_model.transform(graph2vec_embeddings)
        return self.embedding


In [1]:
df1 = pd.read_csv('airforce_channels_anomalous.csv')
df2 = pd.read_csv('random_news_channels.csv')
df_segmentation = pd.concat([df1, df2], axis=0, ignore_index=True)
df_segmentation = df_segmentation.fillna(0)
df_segmentation.head(100)

**Create graphs from gexf_files**

In [2]:
def check_file_in_directory(directory_path, file_name):
    file_path = os.path.join(directory_path, file_name)
    return os.path.isfile(file_path)

graphs = []
gexf_files = []
for i in list(df_segmentation['channel_id']):
    if check_file_in_directory("processed_data", f"{i}.gexf"):
        gexf_files.append('processed_data/'+i+'.gexf')

    else:
        print(f"{i}.gexf does not exist in processed_data")
j=0
graphs = []
graph_ids=[]
for gexf_file in gexf_files:
    graph = nx.read_gexf(gexf_file)
    print(j,graph,gexf_file)
    left_index = 15
    right_index = -5
    left_cut = gexf_file[:left_index]
    right_cut = gexf_file[right_index:]
    graph_id = gexf_file[left_index:right_index]
    graph_ids.append(graph_id)
    graphs.append(graph)
    j+=1

In [3]:
graph2vec = Graph2Vec()
graph2vec.fit(graphs)
graph2vec_umap = Graph2VecUMAP(graph2vec, n_components=4)
embedding = graph2vec_umap.fit_transform(graphs)
print(embedding)

**Optimal number of clusters for K-means clustering (silhouette score)**

In [4]:
def calculate_optimal_clusters(embedding, max_clusters=10):
    silhouette_scores = []
    for n_clusters in range(2, max_clusters + 1):
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        labels = kmeans.fit_predict(embedding)
        score = silhouette_score(embedding, labels)
        silhouette_scores.append((n_clusters, score))
    optimal_n_clusters = max(silhouette_scores, key=lambda x: x[1])[0]
    return optimal_n_clusters, silhouette_scores

def visualize_clusters_3d(embedding, labels, n_clusters):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    colors = plt.cm.get_cmap('tab10', n_clusters)

    for cluster in range(n_clusters):
        cluster_points = embedding[labels == cluster]
        ax.scatter(cluster_points[:, 0], cluster_points[:, 1], cluster_points[:, 2],
                   label=f"Cluster {cluster}", s=50, alpha=0.7)

    ax.set_xlabel('UMAP Dimension 1', fontsize=10, labelpad=0)
    ax.set_ylabel('UMAP Dimension 2', fontsize=10, labelpad=0)
    ax.set_zlabel('UMAP Dimension 3', fontsize=10, labelpad=0)
    ax.set_title(f"KMeans Clustering", fontsize=16, pad=20)

    plt.legend(fontsize=12)
    plt.show()

    max_clusters = 10
optimal_n_clusters, silhouette_scores = calculate_optimal_clusters(embedding, max_clusters)

print(f"Optimal number of clusters: {optimal_n_clusters}")
for n_clusters, score in silhouette_scores:
    print(f"Clusters: {n_clusters}, Silhouette Score: {score:.4f}")

kmeans = KMeans(n_clusters=optimal_n_clusters, random_state=42)
labels = kmeans.fit_predict(embedding)

visualize_clusters_3d(embedding, labels, optimal_n_clusters)

**Optimal number of clusters for Hierarchical Clustering**

In [6]:
def perform_hierarchical_clustering(data, max_clusters=10):
    linkage_matrix = linkage(data, method="ward")
    plt.figure(figsize=(12, 8))
    dendrogram(linkage_matrix)
    plt.title("Dendrogram", fontsize=16)
    plt.xlabel("Channel Index", fontsize=16)
    plt.ylabel("Distance", fontsize=16)
    plt.show()

    silhouette_scores = []
    print("Evaluating Hierarchical clustering silhouette scores:")
    for k in range(2, max_clusters + 1):
        labels = AgglomerativeClustering(n_clusters=k, linkage="ward").fit_predict(data)
        score = silhouette_score(data, labels)
        silhouette_scores.append(score)
        print(f"Number of Clusters: {k}, Silhouette Score: {score:.4f}")

    optimal_k = 2 + np.argmax(silhouette_scores)
    return optimal_k, silhouette_scores

optimal_k, silhouette_scores = perform_hierarchical_clustering(embedding, max_clusters=10)
print(f"Optimal number of clusters based on silhouette scores: {optimal_k}")

Z = linkage(embedding, method='ward')

max_d = 6 
clusters = fcluster(Z, max_d, criterion='distance')
n_clusters = len(set(clusters))

c, coph_dists = cophenet(Z, pdist(embedding))
print("Cophenetic Correlation Coefficient:", c)